# CONSTANTS

**RANDOM_STATE** is a constant that is typically used to initialize the random number generator in a library or algorithm that uses randomness in some aspect of its operation. This means that when the RANDOM_STATE is fixed to a certain value, the algorithm will generate the same random results every time it is run, which can be useful for reproducing results and ensuring result consistency.

In [ ]:
RANDOM_STATE = 0

# LIBRARIES

In [ ]:
import pandas as pd
import warnings

from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import make_pipeline
from joblib import dump
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

# READ FILES

In [ ]:
# Read the training data from the CSV files
X_train = pd.read_csv('../dataset/X_train.csv', usecols = ['TRACK', 'TRUST'])
y_train = pd.read_csv('../dataset/y_train.csv', usecols = ['Blood Bags'])

# MODELING

## - Scores 

Creates a machine learning pipeline that either scales input features and applies a given model or directly applies the given model
- MinMaxScaler: works by subtracting the minimum value in the feature from each observation, and then dividing the result by the range (i.e., the maximum value minus the minimum value). This process ensures that the values in the feature are transformed to a range between 0 and 1.
- BorderlineSMOTE: is an extension of the SMOTE algorithm, which is commonly used for addressing the class imbalance problem in machine learning.

In [ ]:
def create_pipeline(model):
    # Create a pipeline with preprocessing steps and a model
    pipeline = make_pipeline(
        MinMaxScaler(),  # Apply MinMaxScaler for feature scaling
        BorderlineSMOTE(random_state=RANDOM_STATE),  # Apply BorderlineSMOTE for oversampling
        model  # The specified model
    )
    
    # Return the created pipeline
    return pipeline

We will explore two scores in this analysis:

In [ ]:
models = {
    'TRACK': create_pipeline(LogisticRegression(random_state=RANDOM_STATE)),
    'TRUST': create_pipeline(LogisticRegression(random_state=RANDOM_STATE))
}

For logistic regression calibration, we define a search space for its hyperparameters:

In [ ]:
space = {
    'logisticregression__C': Real(1e-2,1e+2),
    'logisticregression__max_iter' : Integer(100,1000),
    'logisticregression__solver': Categorical(['newton-cg','lbfgs','liblinear','sag','saga']),
    'logisticregression__fit_intercept': Categorical([True, False]),
    'logisticregression__class_weight': Categorical(['balanced', None]),     
    'logisticregression__l1_ratio': Real(0,1),
    'borderlinesmote__sampling_strategy': Real(0.85,1.0),
    'borderlinesmote__k_neighbors': Integer(1,10),
    'borderlinesmote__m_neighbors': Integer(1,10)
}

We instantiate a BayesSearchCV object that optimizes the hyperparameters of that specific model

In [ ]:
# Create an empty dictionary to store the optimized models
opt = {}

# Iterate over each item in the 'models' dictionary
for model_name, model in models.items():
    # Create a BayesSearchCV object for hyperparameter optimization
    # using the specified model, search space, and other parameters
    opt[model_name] = BayesSearchCV(
        model,  # The model to optimize
        space,  # The search space for hyperparameters
        n_iter=10,  # Number of parameter settings that are sampled
        cv=RepeatedStratifiedKFold(random_state=RANDOM_STATE),  # Cross-validation strategy
        random_state=RANDOM_STATE,  # Random state for reproducibility
        scoring='roc_auc'  # Scoring metric to optimize
    )

In [ ]:
# Iterate over each item in the 'opt' dictionary
for model_name, model_opt in opt.items():
    # Print the model name followed by a colon
    print(model_name, end=': ')
    
    # Reshape the input data for the current model
    X_train_model = X_train[model_name].values.reshape(-1, 1)
        
    # Fit the model using the selected features and the target variable
    model_opt.fit(X_train_model, y_train)
    
    # Get the best score achieved by the model during cross-validation
    best_score = model_opt.best_score_
    
    # Print the best score
    print(best_score)
    
    # Generate the filename for saving the trained model
    model_filename = '../models/scores/' + model_name + '.joblib'
    
    # Save the trained model to a file
    dump(model_opt.best_estimator_, model_filename)